# Text Analysis on sydneysiege data

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [3]:
df.columns

Index(['src_tweet_id', 'src_user_id', 'src_tweet', 'src_date',
       'reply_tweet_id', 'reply_user_id', 'reply_tweet', 'reply_date',
       'label'],
      dtype='object')

In [4]:
df

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0
1,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544273361334194176,14875436,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:30:56 +0000 2014,0
2,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544275366588588032,14121167,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:38:54 +0000 2014,0
3,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544276615316770816,187877777,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:43:51 +0000 2014,0
4,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544278115589976065,372216479,RT “@nswpolice: A police operation is underway...,Sun Dec 14 23:49:49 +0000 2014,0
...,...,...,...,...,...,...,...,...,...
22770,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522905082540032,2159158525,"“@PoliceAU: Sadly, at this stage, there are tw...",Mon Dec 15 16:02:31 +0000 2014,1
22771,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522916042252289,2159158525,"“@PoliceAU: Sadly, at this stage, there are tw...",Mon Dec 15 16:02:34 +0000 2014,1
22772,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522948850094081,450436433,@PoliceAU no :'(,Mon Dec 15 16:02:42 +0000 2014,1
22773,544521166543548416,87261221,"Sadly, at this stage, there are two confirmed ...",Mon Dec 15 15:55:37 +0000 2014,544522997466267648,450436433,@PoliceAU nonono,Mon Dec 15 16:02:54 +0000 2014,1


In [5]:
src_tw_df = df.iloc[:,[0,1,2,3,8]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df.reset_index(drop=True,inplace=True)
src_tw_df

,src_tweet_id,src_user_id,src_tweet,src_date,label
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,0
1,544268900351766528,16215478,#BREAKING A police operation is underway in th...,Sun Dec 14 23:13:12 +0000 2014,0
2,544274797640044544,7587032,Australian Police Operation Under Way In Sydne...,Sun Dec 14 23:36:38 +0000 2014,0
3,544275574642835456,27964284,Prayers needed for the situation happening in ...,Sun Dec 14 23:39:43 +0000 2014,0
4,544275733607366656,374712154,This is not going to end well in Sydney . Wake...,Sun Dec 14 23:40:21 +0000 2014,0
...,...,...,...,...,...
1168,544520042810200064,442653867,"#sydneysiege is over. 2 confirmed dead, #PrayF...",Mon Dec 15 15:51:09 +0000 2014,1
1169,544520273718812672,100629223,Police confirm that #sydneysiege is finally ov...,Mon Dec 15 15:52:04 +0000 2014,1
1170,544520932405284864,7587032,Seven people reportedly taken away on stretche...,Mon Dec 15 15:54:41 +0000 2014,1
1171,544521141335764992,342894187,BREAKING: CNN: Sydney police storm cafe. 2 con...,Mon Dec 15 15:55:31 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [6]:
import pickle

In [7]:
'''Convert Emojis to Words'''

with open(r'D:\論文\PHEME9\Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    import re
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

In [8]:
def textprocessing(text):
 
    if 'RT' in text.split():      #cut retweet
        rt = text.find('RT')
        text = text[0:rt]  
        
    quo = text.find('“')           #cut quoting part
    quo_end = text.find('”')
    at = text.find('@')
    if quo != -1:                  #if find “
        if quo_end != -1:             # and if there is also ”
            text = text[0: quo:] + text[quo_end+1: :]
        else:
            text = text[0: quo:]
    elif quo_end != -1:
        text = text[0:at:] + text[quo_end+1: : ]
            
   
    text = convert_emojis_to_word(text)   #convert emoji to word
    text = text.lower()           #lower case    
     
    
    text = " ".join([word for word in text.split() if 'http' not in word    #remove url
                                and not word.startswith(('\'@','@','.@'))
                                and word != 'rt'])   
                 
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #remove underscore as well
            w = re.sub('[0-9]+','',w)
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))

    from nltk.tokenize import TweetTokenizer
    tknzr = TweetTokenizer(strip_handles=True)
    text = tknzr.tokenize(text)
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    stop_words=set([a for a in list(stop_words) if re.search("n't|no|not",a) ==None])
    stop_words.add('now')
    stop_words.add('amp')
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(i,'v') for i in text]
    
#     from nltk.stem.porter import PorterStemmer
#     stem = PorterStemmer()
#     text = [stem.stem(i) for i in text]
    
    text = " ".join(text)
    
    return text

In [16]:
print("Source tweet: ", df.src_tweet.iloc[12474])
print("original text: ", df.reply_tweet.iloc[12474])
print("cleaned text: ", textprocessing(df.src_tweet.iloc[12474]))

Source tweet:  #BREAKING: Australian police have named the suspect in the Sydney siege: Islamic leader Sheik Man Haron Monis http://t.co/yoFrfF1AHQ
original text:  Ojo #Uruguay #Chile: Entre los refugiados que recibe, podría haber uno como este: Jeque Islamico Man Haron Monis http://t.co/zKlLbaOEM4”
cleaned text:  break australian police name suspect sydney siege islamic leader sheik man haron monis


In [10]:
quotweet = df[df.reply_tweet.str.find('“') != -1].index
quo_tweet = df[df.reply_tweet.str.find('”') != -1].index
rtt = df[df.reply_tweet.str.find('RT') != -1].index
at =  df[df.reply_tweet.str.find('@') != -1].index

set(quo_tweet).difference(set(rtt)).difference(set(quotweet)).difference(set(at))


{12474}

In [11]:
df[df.reply_tweet.str.find('Ojo') != -1].index

Int64Index([12474], dtype='int64')

### applied on original data (**df**)

In [17]:
src_tw_df['cleaned_src_tw'] = src_tw_df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [18]:
df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_reply_tw
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0,
1,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544273361334194176,14875436,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:30:56 +0000 2014,0,
2,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544275366588588032,14121167,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:38:54 +0000 2014,0,sicken
3,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544276615316770816,187877777,RT @nswpolice: A police operation is underway ...,Sun Dec 14 23:43:51 +0000 2014,0,
4,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544278115589976065,372216479,RT “@nswpolice: A police operation is underway...,Sun Dec 14 23:49:49 +0000 2014,0,


In [19]:
src_tw_df.head()

,src_tweet_id,src_user_id,src_tweet,src_date,label,cleaned_src_tw
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,0,police operation underway martin place sydneys...
1,544268900351766528,16215478,#BREAKING A police operation is underway in th...,Sun Dec 14 23:13:12 +0000 2014,0,break police operation underway heart sydneys ...
2,544274797640044544,7587032,Australian Police Operation Under Way In Sydne...,Sun Dec 14 23:36:38 +0000 2014,0,australian police operation way sydney
3,544275574642835456,27964284,Prayers needed for the situation happening in ...,Sun Dec 14 23:39:43 +0000 2014,0,prayers need situation happen sydney right
4,544275733607366656,374712154,This is not going to end well in Sydney . Wake...,Sun Dec 14 23:40:21 +0000 2014,0,not go end well sydney wake britain come


In [20]:
src_tw_df.cleaned_src_tw.iloc[158]

''

## Sentiment Analysis

In [21]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [22]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on src tweet & reply tweet

In [23]:
src_tw_df['src_sentiment']= src_tw_df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [24]:
df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label,cleaned_reply_tw,reply_sentiment
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,544270066661851136,27735195,“@nswpolice: A police operation is underway in...,Sun Dec 14 23:17:50 +0000 2014,0,,Neutral


In [25]:
src_tw_df.head(1)

,src_tweet_id,src_user_id,src_tweet,src_date,label,cleaned_src_tw,src_sentiment
0,544266241511133186,23041630,A police operation is underway in Martin Place...,Sun Dec 14 23:02:38 +0000 2014,0,police operation underway martin place sydneys...,Negative


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [ ]:
import text2emotion as te

In [ ]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [ ]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emo_te'] = emo

In [26]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-df.csv',index=False)

In [ ]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emo_te']=emo1

In [27]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\sydneysiege-src.csv',index=False)